# Chap4: word2vec の高速化

Embedding layer と nagative sampling を導入して word2vec を完成させる

In [1]:
import os
from pathlib import Path
import sys

sys.path.append(os.path.join(Path().resolve(), 'deep-learning-from-scratch-2'))

## 4.1 word2vec の改良1

- one-hot 表現では語彙数が増えるとベクトルサイズも増え、重みとの積の計算量が増える
  - Embedding layer で対応
- 中間層から出力層への計算と Softmax の計算も語彙数とともに計算量が増大 (Softmax の分母で語彙数分の和を計算する必要がある)
  - negative sampling で対応

### 4.1.1 Embedding layer

- one-hot vector と重み行列の関は実は1に対応する行を抜きだしてるだけ
- この処理を行うのが embedding layer
  - 順伝播は特定の行を抜き出す、逆伝播は特定の行に代入 (加算) する処理になる

In [16]:
from common.layers import Embedding

Embedding??

Init signature: Embedding(W)
Docstring:      <no docstring>
Source:        
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        if GPU:
            np.scatter_add(dW, self.idx, dout)
        else:
            np.add.at(dW, self.idx, dout)
        return None
File:           ~/git/zero-deep-2/deep-learning-from-scratch-2/common/layers.py
Type:           type
Subclasses:     


## 4.2 word2vec の改良2

### 4.2.2

- Negatice sampling のキーとなるアイディアは多値分類を二値分類で近似すること



In [20]:
# 4.2.4 実装
# Embedding と dot を一個にまとめる
from ch04.negative_sampling_layer import EmbeddingDot

EmbeddingDot??

Init signature: EmbeddingDot(W)
Docstring:      <no docstring>
Source:        
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh
File:           ~/git/zero-deep-2/deep-learning-from-scratch-2/ch04/negative_sampling_layer.py
Type:           type
Subclasses:     


In [23]:
# 4.2.7 Negative Sampling の実装
# 二値分類の負例を一部サンプルするのが negative sampling
from ch04.negative_sampling_layer import NegativeSamplingLoss

NegativeSamplingLoss??

Init signature: NegativeSamplingLoss(W, corpus, power=0.75, sample_size=5)
Docstring:      <no docstring>
Source:        
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 正例のフォワード
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 負例の

In [25]:
# 4.3.1
# Embedding Layer と Negative Sampling Layer を使うように改良した CBOW
from ch04.cbow import CBOW

CBOW??

Init signature: CBOW(vocab_size, hidden_size, window_size, corpus)
Docstring:      <no docstring>
Source:        
class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 重みの初期化
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # レイヤの生成
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embeddingレイヤを使用
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # すべての重みと勾配をリストにまとめる
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # メンバ変数に単語の分散表現を設定
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in 

In [27]:
# 4.3.2 CBOW の学習
sys.path.append(os.path.join(Path().resolve(), 'deep-learning-from-scratch-2/ch04'))
# CPU では半日程度かかるのでコメントアウト
#from ch04 import train
#train??

Done
| epoch 1 |  iter 1 / 9295 | time 0[s] | loss 4.16
| epoch 1 |  iter 21 / 9295 | time 1[s] | loss 4.16
| epoch 1 |  iter 41 / 9295 | time 3[s] | loss 4.15
| epoch 1 |  iter 61 / 9295 | time 4[s] | loss 4.12
| epoch 1 |  iter 81 / 9295 | time 5[s] | loss 4.05
| epoch 1 |  iter 101 / 9295 | time 7[s] | loss 3.92
| epoch 1 |  iter 121 / 9295 | time 8[s] | loss 3.77
| epoch 1 |  iter 141 / 9295 | time 10[s] | loss 3.62
| epoch 1 |  iter 161 / 9295 | time 11[s] | loss 3.48
| epoch 1 |  iter 181 / 9295 | time 12[s] | loss 3.37
| epoch 1 |  iter 201 / 9295 | time 14[s] | loss 3.24
| epoch 1 |  iter 221 / 9295 | time 15[s] | loss 3.15
| epoch 1 |  iter 241 / 9295 | time 16[s] | loss 3.09
| epoch 1 |  iter 261 / 9295 | time 18[s] | loss 3.01
| epoch 1 |  iter 281 / 9295 | time 19[s] | loss 2.96
| epoch 1 |  iter 301 / 9295 | time 21[s] | loss 2.92
| epoch 1 |  iter 321 / 9295 | time 22[s] | loss 2.86
| epoch 1 |  iter 341 / 9295 | time 23[s] | loss 2.83
| epoch 1 |  iter 361 / 9295 | time 

KeyboardInterrupt: 

In [28]:
# 4.3.2 CBOW の評価
# 学習が終わってないと pickle がなくてエラーになる
from ch04 import eval
# eval??

FileNotFoundError: [Errno 2] No such file or directory: 'cbow_params.pkl'

## 4.4

- word2vec で得られた単語の分散表現は他の機械学習アプリケーションに用いることができる
- 分散表現の性能評価
  - 単語の類似性は人間が作成した単語類似度と word2vec のコサイン類似度を比較する
  - 類推問題は king:queen = man:? みたいなやつの正解率を測る